In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Split and scale the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

# Define the Neural Network
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(20, 64)
        self.layer_2 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(32)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))
        return x

# Initialize model, loss, and optimizer
model = BinaryClassifier()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
batch_size = 32
for epoch in range(epochs):
    model.train()
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size].reshape(-1, 1)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Evaluation
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            predictions = (test_outputs >= 0.5).float()
            accuracy = (predictions.reshape(-1) == y_test).float().mean()
            print(f'Epoch {epoch}: Test Accuracy = {accuracy:.4f}')

# Final evaluation
model.eval()
with torch.no_grad():
    final_outputs = model(X_test)
    final_predictions = (final_outputs >= 0.5).float()
    final_accuracy = (final_predictions.reshape(-1) == y_test).float().mean()
    print(f'\nFinal Test Accuracy: {final_accuracy:.4f}')

Epoch 0: Test Accuracy = 0.7400
Epoch 10: Test Accuracy = 0.8350
Epoch 20: Test Accuracy = 0.8350
Epoch 30: Test Accuracy = 0.8300
Epoch 40: Test Accuracy = 0.8500
Epoch 50: Test Accuracy = 0.8450
Epoch 60: Test Accuracy = 0.8650
Epoch 70: Test Accuracy = 0.8400
Epoch 80: Test Accuracy = 0.8450
Epoch 90: Test Accuracy = 0.8150

Final Test Accuracy: 0.8450
